In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset
from models import SpeechToTextModel

In [ ]:
from transformers import WhisperProcessor, TrainingArguments, Trainer, AutoTokenizer

In [ ]:
from utils import LibriSpeechDataCollator

In [ ]:
WHISPER_MODEL_NAME = "openai/whisper-base"
LLAMA_MODEL_NAME = "meta-llama/Llama-3.2-3B"

In [ ]:
model = SpeechToTextModel(
    whisper_model_name=WHISPER_MODEL_NAME,
    llama_model_name=LLAMA_MODEL_NAME,
    hidden_dims=[2048, 1024, 2048],
    train_whisper=False
)

In [ ]:
dataset_name = "openslr/librispeech_asr"

In [ ]:
dataset = load_dataset(dataset_name, 'clean', split='train.100')

In [ ]:
processor = WhisperProcessor.from_pretrained(WHISPER_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL_NAME)

In [ ]:
training_args = TrainingArguments(
    output_dir="./v1-checkpoints",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    bf16=True,
    remove_unused_columns=False,
    learning_rate=5e-5,
    report_to="none",
    save_safetensors=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=LibriSpeechDataCollator(processor, tokenizer),
)

In [ ]:
trainer.train()

In [ ]:
model


In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
print(torch.cuda.memory_summary())

In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()
with torch.no_grad():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                del obj
        except:
            pass
torch.cuda.empty_cache()

In [ ]:
inspect.getsource(LibriSpeechDataCollator)

In [ ]:
batch = dataset.select(range(1))
lbdc = LibriSpeechDataCollator(processor, tokenizer)

In [ ]:
input_parameters = lbdc(batch)

In [ ]:
print(input_parameters["input_features"].shape)
print(input_parameters["labels"].shape)
print(input_parameters["input_ids"].shape)

In [ ]:
input_parameters['input_features'] = input_parameters['input_features'].cuda()
input_parameters['labels'] = input_parameters['labels'].cuda()
input_parameters['input_ids'] = input_parameters['input_ids'].cuda()

In [ ]:
model = model.cuda()

In [ ]:
outputs = model.forward(
    input_features=input_parameters['input_features'],
    labels=input_parameters['labels'],
    input_ids=input_parameters['input_ids'],
)

In [ ]:
print(outputs.logits.shape)

In [ ]:
print(outputs.loss)